# Astrabolt - Load Products

This notebook uses **ragstack-ai** and **google-cloud-aiplatform** to connect to Astra DB, create the collections needed, and insert the product catalog with its vector embeddings using gemini multimodal embeeding.

## Install Dependencies, Authenticate, and Create Collections

In [ ]:
!pip install google-cloud-aiplatform ragstack-ai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 940.5/940.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import getpass, os, requests

if "GCP_PROJECT_ID" not in os.environ or True:
  os.environ["GCP_PROJECT_ID"] = getpass.getpass("Provide your GCP Project ID")

if "ASTRA_DB_ENDPOINT" not in os.environ or True:
  os.environ["ASTRA_DB_ENDPOINT"] = getpass.getpass("Provide your Astra DB Endpoint")

if "ASTRA_DB_TOKEN" not in os.environ or True:
  os.environ["ASTRA_DB_TOKEN"] = getpass.getpass("Provide your Astra DB Token")

Provide your GCP Project ID··········
Provide your Astra DB Endpoint··········
Provide your Astra DB Token··········


In [ ]:
from google.colab import auth
from google.cloud import aiplatform

!gcloud config set project {os.getenv("GCP_PROJECT_ID")}

auth.authenticate_user()


Updated property [core/project].


In [ ]:
from astrapy.db import AstraDB
# Initialize our vector db
astra_db = AstraDB(token=os.getenv("ASTRA_DB_TOKEN"), api_endpoint=os.getenv("ASTRA_DB_ENDPOINT"))
collection_descriptions = astra_db.create_collection(collection_name="product_catalog_descriptions", dimension=1408)
collection_images = astra_db.create_collection(collection_name="product_catalog_images", dimension=1408)

## Download Product Catalog

In [ ]:
PRODUCT_CATALOG_URI="https://raw.githubusercontent.com/BestBuyAPIs/open-data-set/master/products.json"

In [ ]:
!wget $PRODUCT_CATALOG_URI

--2023-12-28 03:21:20--  https://raw.githubusercontent.com/BestBuyAPIs/open-data-set/master/products.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39685207 (38M) [text/plain]
Saving to: ‘products.json’

products.json       100%[===================>]  37.85M   167MB/s    in 0.2s    

2023-12-28 03:21:21 (167 MB/s) - ‘products.json’ saved [39685207/39685207]



In [ ]:
import json
with open('products.json', 'r') as f:
    data = json.load(f)
products_slice=data[:1000] #Feel free to modify the size of the dataset as needed

## Create and Load Vector Embeddings
We will first create some helper functions for things such as downloading images, inserting records into Astra DB. And finally we will load the data prom `products_slice` in batches.

In [ ]:
import requests
from PIL import Image
def download_image(image_url): #Downloads images from public bestbuy's URI to local file
  try:
    response = requests.get(image_url)
    response.raise_for_status()  # Raise an exception for error status codes

    filename = image_url.rsplit("/", 1)[-1]
    # Create the folder if it doesn't exist
    folder_path="product_images"
    if not os.path.exists(folder_path):
      os.makedirs(folder_path)
    file_path=f"product_images/{filename}"
    with open(file_path, "wb") as f:
      f.write(response.content)
      return file_path
  except requests.exceptions.HTTPError as err:
    if err.response.status_code == 404:
        print("File not found at the specified URL.")
        return None
    else:
        print("An error occurred:", err)
        return None

In [ ]:
def insert_document(collection,document, verbose=0): #loads the document into the specified collection
  try:
    # add to the AstraDB Vector Database
    collection.insert_one(document)
  except Exception as error:
    # if you've already added this record, skip the error message
    error_info = json.loads(str(error))
    if error_info[0]['errorCode'] == "DOCUMENT_ALREADY_EXISTS" and verbose>0:
      print("Document already exists in the database.  Skipping.")

In [ ]:
from langchain.chat_models import ChatVertexAI

llm = ChatVertexAI(project=os.getenv("GCP_PROJECT_ID"), model_name="gemini-pro-vision", region="uswest-1")


In [ ]:
import tqdm, time
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image

model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")
multimodalembedding_requests_per_minute=120
batch_size=multimodalembedding_requests_per_minute

# Process product catalog in batches of 1000
for batch_start in tqdm.tqdm(range(0, len(products_slice), batch_size), desc="Processing product catalog"):
    batch_products = products_slice[batch_start:batch_start + batch_size]

    # Create embeddings for the batch of products
    t1 = time.perf_counter()
    for product in batch_products:
      filename = product['image'].rsplit("/", 1)[-1]
      import os.path
      if not os.path.exists(f"product_images/{filename}"):
        product["_id"] = product["sku"]
        product["description"] = f'{product["name"]}. {product["description"]}'
        product["main_category"] = product["category"][0]
        filename=download_image(product['image'])
        if filename is not None:
          img = Image.load_from_file(filename)
          embeddings = model.get_embeddings(image=img, contextual_text=product['description'])
        else:
          embeddings = model.get_embeddings(contextual_text=product['description'])
        product["$vector"] = embeddings.text_embedding
        insert_document(collection_descriptions,product)
        if filename is not None:
          product["$vector"] = embeddings.image_embedding
          insert_document(collection_images,product)
    #t2 = time.perf_counter()
    #if t2-t1<60:
    #  time.sleep(60-(t2-t1)) #to stay under GCP's API requests quota


Processing product catalog:   0%|          | 0/84 [00:00<?, ?it/s]

File not found at the specified URL.
File not found at the specified URL.


Processing product catalog:   1%|          | 1/84 [00:01<02:12,  1.59s/it]

File not found at the specified URL.
File not found at the specified URL.


Processing product catalog:   2%|▏         | 2/84 [00:02<01:44,  1.27s/it]

...

Processing product catalog:  98%|█████████▊| 82/84 [2:44:50<05:16, 158.19s/it]

File not found at the specified URL.
File not found at the specified URL.
File not found at the specified URL.
File not found at the specified URL.
File not found at the specified URL.


Processing product catalog:  99%|█████████▉| 83/84 [2:47:40<02:41, 161.86s/it]

File not found at the specified URL.
File not found at the specified URL.


Processing product catalog: 100%|██████████| 84/84 [2:49:05<00:00, 120.78s/it]


## Experiment Querying the Database
We'll run a couple of queries using text, and image as input.

In [ ]:
import json
from vertexai.preview.vision_models import MultiModalEmbeddingModel, Image
from langchain.schema.messages import HumanMessage

model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")

# Embed the similar item
img = Image.load_from_file('2877554_sa.jpg')
embeddings = model.get_embeddings(image=img, contextual_text="")

# Perform the vector search against AstraDB Vector
documents = collection_images.vector_find(
    embeddings.image_embedding,
    limit=3,
)

related_products_csv = "name, image, price, url\n"
for doc in documents:
  related_products_csv += f"{doc['name']}, {doc['image']}, {doc['price']}, {doc['url']},\n"
print(related_products_csv)

name, image, price, url
Griffin Technology - GuitarConnect Cable - Black, http://img.bbystatic.com/BestBuy_US/images/products/1114/1114103_sa.jpg, 29.99, http://www.bestbuy.com/site/griffin-technology-guitarconnect-cable-black/1114103.p?id=1219470055093&skuId=1114103&cmp=RMXCC,
Blue Microphones - Mo-Fi Over-the-Ear Headphones - Gray, http://img.bbystatic.com/BestBuy_US/images/products/1095/1095039_sa.jpg, 349.98, http://www.bestbuy.com/site/blue-microphones-mo-fi-over-the-ear-headphones-gray/1095039.p?id=1219469043846&skuId=1095039&cmp=RMXCC,
Bell'O - TV Stand for Flat-Panel TVs Up to 52" - Cherry, http://img.bbystatic.com/BestBuy_US/images/products/1111/1111949_sa.jpg, 349.99, http://www.bestbuy.com/site/bello-tv-stand-for-flat-panel-tvs-up-to-52-cherry/1111949.p?id=1218220770595&skuId=1111949&cmp=RMXCC,



In [ ]:
embeddings = model.get_embeddings(contextual_text="AudioQuest - Niagara 1200 Low-Z Power Conditioner")

# Perform the vector search against AstraDB Vector
documents = collection_descriptions.vector_find(
    embeddings.text_embedding,
    limit=3,
)

related_products_csv = "name, image, price, url\n"
for doc in documents:
  related_products_csv += f"{doc['name']}, {doc['image']}, {doc['price']}, {doc['url']},\n"
print(related_products_csv)

name, image, price, url
Blueair - SmokeStop Filter for Blueair 200 and 300 Series Air Purifiers - Black, http://img.bbystatic.com/BestBuy_US/images/products/1197/1197082_sa.jpg, 99.99, http://www.bestbuy.com/site/blueair-smokestop-filter-for-blueair-200-and-300-series-air-purifiers-black/1197082.p?id=1218231508373&skuId=1197082CC,
Pro Tools Tier 2 Audio Plug-In for PC and Mac Activation Card - Windows|Mac, http://img.bbystatic.com/BestBuy_US/images/products/1003/1003278_sa.jpg, 299, http://www.bestbuy.com/site/pro-tools-tier-2-audio-plug-in-for-pc-and-mac-activation-card-windowsmac/1003278.p?id=1219460772921&skuId=1003278CC,
Acoustic Rock TrackPak - Mac, http://img.bbystatic.com/BestBuy_US/images/products/1003/1003373_sa.jpg, 29.99, http://www.bestbuy.com/site/acoustic-rock-trackpak-mac/1003373.p?id=1219460754207&skuId=1003373CC,

